In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import xgboost as xgb
from sklearn import preprocessing

In [ ]:
df1 = pd.read_csv("data/train_preliminary/user.csv")
df2 = pd.read_csv("data/train_preliminary/ad.csv")
df3 = pd.read_csv("data/train_preliminary/click_log.csv")
df_tmp = pd.merge(df3, df1, left_on='user_id', right_on='user_id')
df = pd.merge(df_tmp, df2, left_on='creative_id', right_on='creative_id')

In [ ]:
df.head()

In [ ]:
order = ['user_id', 'time', 'creative_id', 'click_times', 'ad_id', 'product_category', 'advertiser_id', 
         'age', 'gender']
df = df[order]
df.head()

In [ ]:
df.shape

In [ ]:
df.describe()

In [ ]:
Y = df[['user_id', 'age', 'gender']]
Y = Y.drop_duplicates(subset=['user_id'], keep='first')
Y = Y.sort_values('user_id')
Y.head()

In [ ]:
Y1_train, Y2_train, Y1_test, Y2_test = Y.iloc[:800000, 1:2], Y.iloc[:800000, 2:3], Y.iloc[800000:, 1:2], Y.iloc[800000:, 2:3]
Y1_train

In [ ]:
# 点击次数总和
X = df3.groupby('user_id')['click_times'].sum()
X

In [ ]:
df["fre_creative_id"] = (df.groupby("user_id")["creative_id"].transform(lambda x: x.value_counts().index[0]))
X_tmp = df.drop_duplicates(subset=['user_id'],keep='first')
X_tmp = X_tmp[['user_id', 'creative_id']]
X = pd.merge(X, X_tmp, left_on='user_id', right_on='user_id')
X

In [ ]:
# 点击最多的类
X = pd.merge(X, df2[['creative_id', 'product_category']], left_on='creative_id', right_on='creative_id')
X = X.drop('creative_id', axis = 1)
X_tmp = pd.get_dummies(X['product_category'], prefix='category')
X = pd.concat([X.drop('product_category', axis = 1), X_tmp], axis=1)
X = X.sort_values('user_id')
X

In [ ]:
X_ready = X.drop(['user_id'], axis = 1)
X_ready = preprocessing.MinMaxScaler().fit_transform(X_ready)
X_train, X_test = X_ready[:800000, :], X_ready[800000:, :]
model1 = xgb.XGBClassifier()
model1.fit(X_train, Y2_train)
model1.score(X_train, Y2_train)

In [ ]:
X_tmp1 = df3.groupby('user_id')['click_times'].mean()
X_tmp2 = df3.groupby('user_id')['click_times'].std()

In [ ]:
# 点击次数的平均值和集中程度
X = pd.merge(X, X_tmp1, left_on='user_id', right_on='user_id')
X = pd.merge(X, X_tmp2, left_on='user_id', right_on='user_id')
X

In [ ]:
X_ready = X.drop(['user_id'], axis = 1)
X_ready = preprocessing.MinMaxScaler().fit_transform(X_ready)
X_train, X_test = X_ready[:800000, :], X_ready[800000:, :]
model1 = xgb.XGBClassifier()
model1.fit(X_train, Y2_train)
model1.score(X_train, Y2_train)

In [ ]:
# 点击时间分布情况
X_tmp = df[['user_id', 'time']].groupby('user_id')['time'].std()
X = pd.merge(X, X_tmp, left_on='user_id', right_on='user_id')
X

In [ ]:
X_ready = X.drop(['user_id'], axis = 1)
X_ready = preprocessing.MinMaxScaler().fit_transform(X_ready)
X_train, X_test = X_ready[:800000, :], X_ready[800000:, :]
model1 = xgb.XGBClassifier()
model1.fit(X_train, Y2_train)
model1.score(X_train, Y2_train)

In [ ]:
# creave_id 以取模方式进入特征
mo = 200
X_tmp = df.drop_duplicates(subset=['user_id'],keep='first')
X_tmp = X_tmp[['user_id', 'creative_id']]
X_tmp['creative_id'] = X_tmp['creative_id'] % mo
X_tmp

In [ ]:
X_tmp = X_tmp.drop_duplicates(subset=['user_id'], keep='first')
X_tmp = X_tmp.sort_values('user_id')
X_tmp = pd.get_dummies(X_tmp['creative_id'], prefix='creative_id')
X_tmp = X_tmp.reset_index(drop=True)

In [ ]:
X = pd.concat([X, X_tmp], axis = 1)
X

In [ ]:
X_ready = X.drop(['user_id'], axis = 1)
X_ready = preprocessing.MinMaxScaler().fit_transform(X_ready)
X_train, X_test = X_ready[:800000, :], X_ready[800000:, :]
model1 = xgb.XGBClassifier()
model1.fit(X_train, Y2_train)
model1.score(X_train, Y2_train)